In [ ]:
from pathlib import Path
from loguru import logger
datadir = Path('data/raw').resolve()
datadir.exists()

# Qanon

QAnon emerged on 4chan in 2017 as anonymous posts ("Q drops") claiming to be from a government insider exposing a supposed deep state conspiracy. The poster(s) alleged that Donald Trump was fighting a secret war against corrupt elites.
The movement gained significant following and influenced real-world events, including:

- The January 6th Capitol riot, where many participants expressed QAnon beliefs
- Multiple incidents of violence/threats by believers
- Spread of misinformation during COVID-19 pandemic

<img src="img/Qanon.png" width=450 height=400 />

We will attempt to analyze authorship, using basic NLP techniques. This is valuable as it could provide insights into the origins and evolution of this influential movement.

So lets preprocess the data, by adding and timebins based on the image shown above.

In [ ]:
from pandas import json_normalize
import json

datafile = datadir / 'posts.json'
with datafile.open() as f:
    df = json_normalize(json.load(f)["posts"], sep="_")

In [ ]:
df.head()

In [ ]:
from datetime import datetime
import pandas as pd
import re

def bin_time(time):
    if time < datetime(2017, 12, 1):
        return 0
    elif time < datetime(2018, 1, 1):
        return 1
    elif time < datetime(2018, 8, 10):
        return 2
    elif time < datetime(2019, 8, 1):
        return 3
    else:
        return 4

def remove_url(text):
    return re.sub(r'^https?:\/\/.*[\r\n]*', '', text)

df["time"] = df["post_metadata_time"].apply(pd.to_datetime, unit="s")
df["bintime"] = df["time"].apply(lambda x : bin_time(x))
df["text"] = df["text"].apply(lambda x : str(x).replace("\n", " "))
df["text"] = df["text"].apply(lambda x : remove_url(x))
df["text"] = df["text"].apply(lambda x : x.lower())
df['size'] = df['text'].apply(lambda x : len(str(x)))
df = df[df["size"] > 50]
df.reset_index(inplace=True, drop=True)

In [ ]:
processeddir = Path("data/processed").resolve()
if not processeddir.exists():
    logger.info(f"Creating directory {processeddir}")
    processeddir.mkdir(parents=True)
outputfile = processeddir / Path("posts.parquet")
df.to_parquet(outputfile)